In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import io

In [2]:
class AdmissionPredictor(nn.Module):
    def __init__(self, input_size):
        super(AdmissionPredictor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

In [ ]:
def load_data():
    # Read the CSV file
    df = pd.read_csv('/')
    return df

In [31]:
def train_model(df):
    # Prepare features
    categorical_cols = ['UG_institute_tier', 'UG_branch', 'GATE_paper', 'Category', 
                       'university_name', 'dept_specialization', 'university_tier', 'location_state']
    
    numerical_cols = ['UG_percentage', 'Year_of_passing', 'GATE_score', 'GATE_rank', 
                     'Work_exp_years', 'previous_cutoff_gen', 'previous_cutoff_obc',
                     'previous_cutoff_sc', 'previous_cutoff_st', 'min_UG_required', 
                     'difference_from_cutoff']
    
    # Encode categorical variables
    label_encoders = {}
    df_encoded = df.copy()
    
    for col in categorical_cols:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Prepare features and target
    feature_cols = categorical_cols + numerical_cols
    X = df_encoded[feature_cols].values
    y = df_encoded['admitted_flag'].values
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1)
    
    # Initialize model
    model = AdmissionPredictor(X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Training
    epochs = 1500
    batch_size = 64
    
    for epoch in range(epochs):
        model.train()
        for i in range(0, len(X_train_tensor), batch_size):
            batch_X = X_train_tensor[i:i+batch_size]
            batch_y = y_train_tensor[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        test_predictions = (test_outputs > 0.5).float()
        accuracy = (test_predictions == y_test_tensor).float().mean()
    
    return model, scaler, label_encoders, feature_cols, accuracy.item()


In [32]:
df = load_data()

In [33]:
model, scaler, label_encoders, feature_cols, accuracy = train_model(df)

In [34]:
print(f"RNN Model trained with accuracy: {accuracy*100:.2f}%")

RNN Model trained with accuracy: 91.30%
